In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api
import scipy
import statsmodels
from sklearn.base import clone
import doubleml as dml
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LassoCV, Lasso
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000

In [2]:
#load data
df_log=pd.read_csv("dataframe_transformed_4_log.csv")
# df_log.drop("Unnamed: 0", axis=1, inplace=True)

In [3]:
# short data transformation:

# #for models that cannot handle NaN values: need to replace missing/zero observations:
# #largest value of column in stead of -infinity (same for both) to impute: because the week before was a promo (therefore obs = 0, therefore log(0)=-inf)
# maxt1= df_log[df_log['L_PRODUCT_ORDER_AMT_T_1']>df_log['L_PRODUCT_ORDER_AMT_T_1'].min()]['L_PRODUCT_ORDER_AMT_T_1'].max()
# maxt2=df_log[df_log['L_PRODUCT_ORDER_AMT_T_2']>df_log['L_PRODUCT_ORDER_AMT_T_2'].min()]['L_PRODUCT_ORDER_AMT_T_2'].max()
# #create new column with imputed value in stead of -infty
# df_log['L_PRODUCT_ORDER_AMT_T_2_m']=df_log['L_PRODUCT_ORDER_AMT_T_2'].copy()
# df_log['L_PRODUCT_ORDER_AMT_T_2_m'].replace([np.inf, -np.inf, np.nan], maxt1, inplace=True)
# df_log['L_PRODUCT_ORDER_AMT_T_1_m']=df_log['L_PRODUCT_ORDER_AMT_T_1'].copy()
# df_log['L_PRODUCT_ORDER_AMT_T_1_m'].replace([np.inf, -np.inf, np.nan], maxt2, inplace=True)
# include dummy for holidays and other special dates that can explain irregular demand


listofHolidays=["2020-12-28",
                "2021-01-04",
                "2021-03-29",
                "2021-05-03",
                "2021-07-26",
                "2021-08-02",
                "2021-08-09",
                "2021-08-16",
                "2021-12-27",
                "2022-01-03",
                "2022-04-11",
                "2022-05-02",
                "2022-07-25",
                "2022-08-01",
                "2022-08-08",
                "2022-08-15",
                "2022-12-26"
                ]
df_log['HOLIDAY']=(df_log['WEEK_START_DATE'].isin(listofHolidays))*1

# transformed the week number variable -> but decided that you want non-log-transformed version of this variable.
# non-log-transformed week number because:do not need interpretation of log-log which would be: "percentage increase in week number leads to percentage change in demand"
# but we want interpretation of absolute increase in weeks (inflation measure) on --> effect on % demand
df_log['WEEK_NR']=np.round(np.exp(df_log['L_WEEK_NR']))

In [4]:
#xgboost can handle np.nan (instead of -inf): 
df=df_log.copy()
df.replace([-np.inf], np.nan, inplace=True)

Create variable to filter on: (from dummy to categorical)

In [5]:
df.columns

Index(['KEY_ARTICLE', 'KEY_WEEK', 'L_PRODUCT_ORDER_AMT', 'CONST',
       'L_PRODUCT_ORDER_AMT_T_1', 'L_AVG_SELL_PRICE_T_1',
       'L_PRODUCT_ORDER_AMT_T_2', 'L_AVG_SELL_PRICE_T_2', 'PPL2', 'PPL3',
       'YEAR_CALENDAR_WEEK', 'WEEK_START_DATE', 'ARTICLE_NAME', 'ARTICLE_ID',
       'ARTICLE_TIER_BETTER', 'ARTICLE_TIER_GOOD',
       'ART_BRAND_TIER_PRICE_ENTRY', 'ART_BRAND_TIER_PRIVATE_LABEL',
       'PACKAGING_BOX', 'PACKAGING_CAN', 'PACKAGING_PACK',
       'ARTICLE_CAT_2_DRINKPAKJES', 'ARTICLE_CAT_2_FRUITDRANK',
       'ARTICLE_CAT_2_IJSTHEE', 'ARTICLE_CAT_2_LIMONADE_SIROPEN',
       'ARTICLE_CAT_2_SAPPEN_SMOOTHIES', 'ARTICLE_CAT_2_SINAS_LEMON_CASSIS',
       'ARTICLE_CAT_2_SPECIAAL_FRIS', 'ARTICLE_CAT_2_SPORT_ENERGYDRINK',
       'ARTICLE_CAT_2_WATER', 'PROMO_DUMMY', 'PPL1', 'POSTPPL1', 'POSTPPL2',
       'POSTPPL3', 'L_AVG_SELL_PRICE', 'L_AVG_SELL_PRICE_SQ',
       'L_AVG_PURCHASE_PRICE', 'L_AVG_PURCHASE_PRICE_SQ', 'L_AVG_HIGH_TEMP',
       'L_AVG_HIGH_TEMP_SQ', 'L_TOTAL_ORDER_AMT',

In [6]:
# Would be easiest to use categorical variable: for now memory is full so manually transform dummy variables back to categorical variables
# Combine the dummy variables into a single categorical variable
df['ART_CAT_2'] = df.apply(lambda x: 'Drinkpakjes' if x['ARTICLE_CAT_2_DRINKPAKJES'] == 1 else
                                 ('Fruitdrank' if x['ARTICLE_CAT_2_FRUITDRANK'] == 1 else
                                 ('Ijsthee' if x['ARTICLE_CAT_2_IJSTHEE'] == 1 else
                                 ('Limonade & siropen' if x['ARTICLE_CAT_2_LIMONADE_SIROPEN'] == 1 else
                                 ('Sappen & smoothies' if x['ARTICLE_CAT_2_SAPPEN_SMOOTHIES'] == 1 else
                                 ('Sinas, Lemon & Cassis' if x['ARTICLE_CAT_2_SINAS_LEMON_CASSIS'] == 1 else
                                 ('Speciaal fris' if x['ARTICLE_CAT_2_SPECIAAL_FRIS'] == 1 else
                                 ('Sport- & energydrink' if x['ARTICLE_CAT_2_SPORT_ENERGYDRINK'] == 1 else
                                 ('Water' if x['ARTICLE_CAT_2_WATER'] == 1 else 'Cola')))))))), axis=1)

In [8]:
#create data partition you will use in methods

x_lasso0 = df[['ART_CAT_2',
       'CONST','ARTICLE_TIER_BETTER', 'ARTICLE_TIER_GOOD',
       'ART_BRAND_TIER_PRICE_ENTRY', 'ART_BRAND_TIER_PRIVATE_LABEL',
       'PACKAGING_BOX', 'PACKAGING_CAN', 'PACKAGING_PACK',
       'ARTICLE_CAT_2_DRINKPAKJES', 'ARTICLE_CAT_2_FRUITDRANK',
       'ARTICLE_CAT_2_IJSTHEE', 'ARTICLE_CAT_2_LIMONADE_SIROPEN',
       'ARTICLE_CAT_2_SAPPEN_SMOOTHIES', 'ARTICLE_CAT_2_SINAS_LEMON_CASSIS',
       'ARTICLE_CAT_2_SPECIAAL_FRIS', 'ARTICLE_CAT_2_SPORT_ENERGYDRINK',
       'ARTICLE_CAT_2_WATER', 
       'PROMO_DUMMY', 
       'PPL2','PPL3', 'POSTPPL1', 'POSTPPL2','POSTPPL3', 
       'L_AVG_HIGH_TEMP',#'L_TOTAL_ORDER_AMT',
       'L_NR_ARTICLES_IN_CAT', 
       'L_NR_ARTICLES_IN_CAT_2', # 'L_WEEK_NR',
       'AVG_UNAVAILABILITY_PERC',
       'ART_CONTENT_VOLUME', 'ART_IS_MULTIPACK', 
       # 'L_PRODUCT_ORDER_AMT_T_2_m','L_PRODUCT_ORDER_AMT_T_1_m' ,
       # 'L_PRODUCT_ORDER_AMT_T_1', 'L_PRODUCT_ORDER_AMT_T_2',
       #'L_AVG_PURCHASE_PRICE',
       'HOLIDAY','WEEK_NR'
       ]]
x_lasso=x_lasso0.drop('ART_CAT_2',axis=1)

y_lasso = df['L_PRODUCT_ORDER_AMT']
p_lasso = df['L_AVG_SELL_PRICE']
z_lasso = df['L_AVG_PURCHASE_PRICE']

naiveset_lasso=x_lasso0.join(p_lasso.to_frame()) 
dmlset_lasso=x_lasso0.join(y_lasso.to_frame().join(p_lasso.to_frame().join(z_lasso.to_frame()))) #needed in dml algortihm

Test set size smaller in this case: as datasets are smaller

In [9]:
x_lasso0_train, x_lasso0_test,x_lasso_train, x_lasso_test, y_lasso_train, y_lasso_test, p_lasso_train, p_lasso_test, z_lasso_train, z_lasso_test, naiveset_lasso_train, naiveset_lasso_test, dmlset_lasso_train, dmlset_lasso_test = train_test_split(x_lasso0,x_lasso,y_lasso,p_lasso,z_lasso,naiveset_lasso, dmlset_lasso,random_state=44, test_size=0.1) #in case want to see how first stages perform

# Naive methods

OLS

In [10]:
from linearmodels.iv import IV2SLS

# Define what column to filter on
subset_col='ART_CAT_2'

# Get the unique values of the column that we want to use for dividing the subsets
unique_values = dmlset_lasso_train[subset_col].unique()
# Store the results of each subset in a dictionary
results1a = {}
for value in unique_values:
    print("Running for", value,"...")
    subset = dmlset_lasso_train[dmlset_lasso_train[subset_col] == value]
    res_ols = IV2SLS(dmlset_lasso_train['L_PRODUCT_ORDER_AMT'], dmlset_lasso_train.drop(['ART_CAT_2','L_PRODUCT_ORDER_AMT','L_AVG_PURCHASE_PRICE'],axis=1), None, None).fit(cov_type="unadjusted")
    result = res_ols
    # Store the result of the model operation in the dictionary
    results1a[value] = result

# Print all the results
for key, value in results1a.items():
    print(f"Result for value {key}: {value}")

Running for Fruitdrank ...
Running for Cola ...
Running for Limonade & siropen ...
Running for Water ...
Running for Speciaal fris ...
Running for Sappen & smoothies ...
Running for Drinkpakjes ...
Running for Sinas, Lemon & Cassis ...
Running for Ijsthee ...
Running for Sport- & energydrink ...
Result for value Fruitdrank:                              OLS Estimation Summary                            
Dep. Variable:     L_PRODUCT_ORDER_AMT   R-squared:                      0.3533
Estimator:                         OLS   Adj. R-squared:                 0.3530
No. Observations:                61119   F-statistic:                  3.34e+04
Date:                 Thu, Mar 16 2023   P-value (F-stat)                0.0000
Time:                         10:59:47   Distribution:                 chi2(31)
Cov. Estimator:             unadjusted                                         
                                                                               
                                  

TSLS

In [11]:
from linearmodels.iv import IV2SLS

# Define what column to filter on
subset_col='ART_CAT_2'

# Get the unique values of the column that we want to use for dividing the subsets
unique_values = dmlset_lasso_train[subset_col].unique()
# Store the results of each subset in a dictionary
results1b = {}
for value in unique_values:
    print("Running for", value,"...")
    subset = dmlset_lasso_train[dmlset_lasso_train[subset_col] == value]
    res_ols = IV2SLS(dmlset_lasso_train['L_PRODUCT_ORDER_AMT'], dmlset_lasso_train.drop(['ART_CAT_2','L_PRODUCT_ORDER_AMT','L_AVG_PURCHASE_PRICE','L_AVG_SELL_PRICE'],axis=1), dmlset_lasso_train['L_AVG_SELL_PRICE'], dmlset_lasso_train['L_AVG_PURCHASE_PRICE']).fit(cov_type="unadjusted")
    result = res_ols.summary
    # Store the result of the model operation in the dictionary
    results1b[value] = result

# Print all the results
for key, value in results1b.items():
    print(f"Result for value {key}: {value}")

Running for Fruitdrank ...
Running for Cola ...
Running for Limonade & siropen ...
Running for Water ...
Running for Speciaal fris ...
Running for Sappen & smoothies ...
Running for Drinkpakjes ...
Running for Sinas, Lemon & Cassis ...
Running for Ijsthee ...
Running for Sport- & energydrink ...
Result for value Fruitdrank:                            IV-2SLS Estimation Summary                          
Dep. Variable:     L_PRODUCT_ORDER_AMT   R-squared:                      0.3529
Estimator:                     IV-2SLS   Adj. R-squared:                 0.3526
No. Observations:                61119   F-statistic:                 3.217e+04
Date:                 Thu, Mar 16 2023   P-value (F-stat)                0.0000
Time:                         11:00:04   Distribution:                 chi2(31)
Cov. Estimator:             unadjusted                                         
                                                                               
                                  

# DML-PLIV

Lasso

In [12]:
lasso_l = LassoCV(fit_intercept=True, cv=10)
lasso_m = LassoCV(fit_intercept=True, cv=10)
lasso_r = LassoCV(fit_intercept=True, cv=10)

In [13]:
# Define what column to filter on
subset_col='ART_CAT_2'

# Get the unique values of the column that we want to use for dividing the subsets
unique_values = dmlset_lasso_train[subset_col].unique()
# Store the results of each subset in a dictionary
results2 = {}
for value in unique_values:
    print("Running for", value,"...")
    subset = dmlset_lasso_train[dmlset_lasso_train[subset_col] == value]
    # Perform the model operation on the subset
    obj_dml_data = dml.DoubleMLData(subset.drop('ART_CAT_2',axis=1), y_col='L_PRODUCT_ORDER_AMT', d_cols='L_AVG_SELL_PRICE', z_cols='L_AVG_PURCHASE_PRICE',  force_all_x_finite=True)
    #print(obj_dml_data)
    dml_pliv_obj = dml.DoubleMLPLIV(obj_dml_data, lasso_l,lasso_m,lasso_r)
    print(dml_pliv_obj.fit())
    result = dml_pliv_obj.coef
    # Store the result of the model operation in the dictionary
    results2[value] = result

# Print all the results
for key, value in results2.items():
    print(f"Result for value {key}: {value}")

Running for Fruitdrank ...
================== DoubleMLPLIV Object ==================

------------------ Data summary      ------------------
Outcome variable: L_PRODUCT_ORDER_AMT
Treatment variable(s): ['L_AVG_SELL_PRICE']
Covariates: ['CONST', 'ARTICLE_TIER_BETTER', 'ARTICLE_TIER_GOOD', 'ART_BRAND_TIER_PRICE_ENTRY', 'ART_BRAND_TIER_PRIVATE_LABEL', 'PACKAGING_BOX', 'PACKAGING_CAN', 'PACKAGING_PACK', 'ARTICLE_CAT_2_DRINKPAKJES', 'ARTICLE_CAT_2_FRUITDRANK', 'ARTICLE_CAT_2_IJSTHEE', 'ARTICLE_CAT_2_LIMONADE_SIROPEN', 'ARTICLE_CAT_2_SAPPEN_SMOOTHIES', 'ARTICLE_CAT_2_SINAS_LEMON_CASSIS', 'ARTICLE_CAT_2_SPECIAAL_FRIS', 'ARTICLE_CAT_2_SPORT_ENERGYDRINK', 'ARTICLE_CAT_2_WATER', 'PROMO_DUMMY', 'PPL2', 'PPL3', 'POSTPPL1', 'POSTPPL2', 'POSTPPL3', 'L_AVG_HIGH_TEMP', 'L_NR_ARTICLES_IN_CAT', 'L_NR_ARTICLES_IN_CAT_2', 'AVG_UNAVAILABILITY_PERC', 'ART_CONTENT_VOLUME', 'ART_IS_MULTIPACK', 'HOLIDAY', 'WEEK_NR']
Instrument variable(s): ['L_AVG_PURCHASE_PRICE']
No. Observations: 12313

------------------ S

Rfor:

In [14]:
Learnerml_l= RandomForestRegressor(max_features=0.65, n_estimators=500)
Learnerml_m= RandomForestRegressor(max_features=0.65, n_estimators=500)
Learnerml_r= RandomForestRegressor(max_features=0.65, n_estimators=500)

In [15]:
# Define what column to filter on
subset_col='ART_CAT_2'

# Get the unique values of the column that we want to use for dividing the subsets
unique_values = dmlset_lasso_train[subset_col].unique()
# Store the results of each subset in a dictionary
results2 = {}
for value in unique_values:
    print("Running for", value,"...")
    subset = dmlset_lasso_train[dmlset_lasso_train[subset_col] == value]
    # Perform the model operation on the subset
    obj_dml_data = dml.DoubleMLData(subset.drop('ART_CAT_2',axis=1), y_col='L_PRODUCT_ORDER_AMT', d_cols='L_AVG_SELL_PRICE', z_cols='L_AVG_PURCHASE_PRICE',  force_all_x_finite=True)
    #print(obj_dml_data)
    dml_pliv_obj = dml.DoubleMLPLIV(obj_dml_data, Learnerml_l, Learnerml_m, Learnerml_r)
    print(dml_pliv_obj.fit())
    result = dml_pliv_obj.coef
    # Store the result of the model operation in the dictionary
    results2[value] = result

# Print all the results
for key, value in results2.items():
    print(f"Result for value {key}: {value}")

Running for Fruitdrank ...
================== DoubleMLPLIV Object ==================

------------------ Data summary      ------------------
Outcome variable: L_PRODUCT_ORDER_AMT
Treatment variable(s): ['L_AVG_SELL_PRICE']
Covariates: ['CONST', 'ARTICLE_TIER_BETTER', 'ARTICLE_TIER_GOOD', 'ART_BRAND_TIER_PRICE_ENTRY', 'ART_BRAND_TIER_PRIVATE_LABEL', 'PACKAGING_BOX', 'PACKAGING_CAN', 'PACKAGING_PACK', 'ARTICLE_CAT_2_DRINKPAKJES', 'ARTICLE_CAT_2_FRUITDRANK', 'ARTICLE_CAT_2_IJSTHEE', 'ARTICLE_CAT_2_LIMONADE_SIROPEN', 'ARTICLE_CAT_2_SAPPEN_SMOOTHIES', 'ARTICLE_CAT_2_SINAS_LEMON_CASSIS', 'ARTICLE_CAT_2_SPECIAAL_FRIS', 'ARTICLE_CAT_2_SPORT_ENERGYDRINK', 'ARTICLE_CAT_2_WATER', 'PROMO_DUMMY', 'PPL2', 'PPL3', 'POSTPPL1', 'POSTPPL2', 'POSTPPL3', 'L_AVG_HIGH_TEMP', 'L_NR_ARTICLES_IN_CAT', 'L_NR_ARTICLES_IN_CAT_2', 'AVG_UNAVAILABILITY_PERC', 'ART_CONTENT_VOLUME', 'ART_IS_MULTIPACK', 'HOLIDAY', 'WEEK_NR']
Instrument variable(s): ['L_AVG_PURCHASE_PRICE']
No. Observations: 12313

------------------ S

XGBoost

Create data set again - now the NaN values can be used so other columns are selected

In [16]:
x_xgb0 = df[['ART_CAT_2',
       'CONST','ARTICLE_TIER_BETTER', 'ARTICLE_TIER_GOOD',
       'ART_BRAND_TIER_PRICE_ENTRY', 'ART_BRAND_TIER_PRIVATE_LABEL',
       'PACKAGING_BOX', 'PACKAGING_CAN', 'PACKAGING_PACK',
       'ARTICLE_CAT_2_DRINKPAKJES', 'ARTICLE_CAT_2_FRUITDRANK',
       'ARTICLE_CAT_2_IJSTHEE', 'ARTICLE_CAT_2_LIMONADE_SIROPEN',
       'ARTICLE_CAT_2_SAPPEN_SMOOTHIES', 'ARTICLE_CAT_2_SINAS_LEMON_CASSIS',
       'ARTICLE_CAT_2_SPECIAAL_FRIS', 'ARTICLE_CAT_2_SPORT_ENERGYDRINK',
       'ARTICLE_CAT_2_WATER', 
       'PROMO_DUMMY', 
       'PPL2','PPL3', 'POSTPPL1', 'POSTPPL2','POSTPPL3', 
       'L_AVG_HIGH_TEMP',#'L_TOTAL_ORDER_AMT',
       'L_NR_ARTICLES_IN_CAT', 
       'L_NR_ARTICLES_IN_CAT_2', # 'L_WEEK_NR',
       'AVG_UNAVAILABILITY_PERC',
       'ART_CONTENT_VOLUME', 'ART_IS_MULTIPACK',
    #    'L_PRODUCT_ORDER_AMT_T_2_m','L_PRODUCT_ORDER_AMT_T_1_m' ,
    #    'L_PRODUCT_ORDER_AMT_T_1', 'L_PRODUCT_ORDER_AMT_T_2',
       #'L_AVG_PURCHASE_PRICE',
       'HOLIDAY','WEEK_NR'
       ]]
x_xgb=x_xgb0.drop('ART_CAT_2',axis=1)

y_xgb = df['L_PRODUCT_ORDER_AMT']
p_xgb = df['L_AVG_SELL_PRICE']
z_xgb = df['L_AVG_PURCHASE_PRICE']

naiveset_xgb=x_xgb0.join(p_xgb.to_frame()) 
dmlset_xgb=x_xgb0.join(y_xgb.to_frame().join(p_xgb.to_frame().join(z_xgb.to_frame()))) #needed in dml algortihm

Test set size smaller in this case: as datasets are smaller

In [17]:
x_xgb0_train, x_xgb0_test,x_xgb_train, x_xgb_test, y_xgb_train, y_xgb_test, p_xgb_train, p_xgb_test, z_xgb_train, z_xgb_test, naiveset_xgb_train, naiveset_xgb_test, dmlset_xgb_train, dmlset_xgb_test = train_test_split(x_xgb0,x_xgb,y_xgb,p_xgb,z_xgb,naiveset_xgb, dmlset_xgb,random_state=44, test_size=0.1) #in case want to see how first stages perform

In [18]:
#Initialize XGBoost (Parameters Optimized Elsewhere)
xgbr_l=xgb.XGBRegressor(colsample_bytree= 1, gamma= 0, learning_rate= 0.2, max_depth= None, n_estimators= 500).fit(x_xgb_train,y_xgb_train)
xgbr_m=xgb.XGBRegressor(colsample_bytree= 1, gamma= 0, learning_rate= 0.2, max_depth= None, n_estimators= 500).fit(x_xgb_train,z_xgb_train)
xgbr_r=xgb.XGBRegressor(colsample_bytree= 1, gamma= 0, learning_rate= 0.2, max_depth= None, n_estimators= 500).fit(x_xgb_train,p_xgb_train)


Second stage: (which will be executed for each subset)

In [19]:
# Define what column to filter on
subset_col='ART_CAT_2'

# Get the unique values of the column that we want to use for dividing the subsets
unique_values = dmlset_xgb_train[subset_col].unique()
# Store the results of each subset in a dictionary
results1 = {}
for value in unique_values:
    print("Running for", value,"...")
    subset = dmlset_xgb_train[dmlset_xgb_train[subset_col] == value]
    # Perform the model operation on the subset
    obj_dml_data = dml.DoubleMLData(subset.drop('ART_CAT_2',axis=1), y_col='L_PRODUCT_ORDER_AMT', d_cols='L_AVG_SELL_PRICE', z_cols='L_AVG_PURCHASE_PRICE',  force_all_x_finite='allow-nan')
    #print(obj_dml_data)
    dml_pliv_obj = dml.DoubleMLPLIV(obj_dml_data, xgbr_l, xgbr_m, xgbr_r)
    print(dml_pliv_obj.fit())
    result = dml_pliv_obj.coef
    # Store the result of the model operation in the dictionary
    results1[value] = result

# Print all the results
for key, value in results1.items():
    print(f"Result for value {key}: {value}")

Running for Fruitdrank ...
================== DoubleMLPLIV Object ==================

------------------ Data summary      ------------------
Outcome variable: L_PRODUCT_ORDER_AMT
Treatment variable(s): ['L_AVG_SELL_PRICE']
Covariates: ['CONST', 'ARTICLE_TIER_BETTER', 'ARTICLE_TIER_GOOD', 'ART_BRAND_TIER_PRICE_ENTRY', 'ART_BRAND_TIER_PRIVATE_LABEL', 'PACKAGING_BOX', 'PACKAGING_CAN', 'PACKAGING_PACK', 'ARTICLE_CAT_2_DRINKPAKJES', 'ARTICLE_CAT_2_FRUITDRANK', 'ARTICLE_CAT_2_IJSTHEE', 'ARTICLE_CAT_2_LIMONADE_SIROPEN', 'ARTICLE_CAT_2_SAPPEN_SMOOTHIES', 'ARTICLE_CAT_2_SINAS_LEMON_CASSIS', 'ARTICLE_CAT_2_SPECIAAL_FRIS', 'ARTICLE_CAT_2_SPORT_ENERGYDRINK', 'ARTICLE_CAT_2_WATER', 'PROMO_DUMMY', 'PPL2', 'PPL3', 'POSTPPL1', 'POSTPPL2', 'POSTPPL3', 'L_AVG_HIGH_TEMP', 'L_NR_ARTICLES_IN_CAT', 'L_NR_ARTICLES_IN_CAT_2', 'AVG_UNAVAILABILITY_PERC', 'ART_CONTENT_VOLUME', 'ART_IS_MULTIPACK', 'HOLIDAY', 'WEEK_NR']
Instrument variable(s): ['L_AVG_PURCHASE_PRICE']
No. Observations: 12313

------------------ S

# DML-PLR


Lasso

In [20]:
# Define what column to filter on
subset_col='ART_CAT_2'

# Get the unique values of the column that we want to use for dividing the subsets
unique_values = dmlset_lasso_train[subset_col].unique()
# Store the results of each subset in a dictionary
results4 = {}
for value in unique_values:
    print("Running for", value,"...")
    subset = dmlset_lasso_train[dmlset_lasso_train[subset_col] == value]
    # Perform the model operation on the subset
    obj_dml_data = dml.DoubleMLData(subset.drop(['ART_CAT_2','L_AVG_PURCHASE_PRICE'],axis=1), y_col='L_PRODUCT_ORDER_AMT', d_cols='L_AVG_SELL_PRICE', force_all_x_finite=True)
    dml_pliv_obj = dml.DoubleMLPLR(obj_dml_data, lasso_l,lasso_r)
    print(dml_pliv_obj.fit())
    result = dml_pliv_obj.coef
    # Store the result of the model operation in the dictionary
    results4[value] = result

# Print all the results
for key, value in results4.items():
    print(f"Result for value {key}: {value}")

Running for Fruitdrank ...
================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: L_PRODUCT_ORDER_AMT
Treatment variable(s): ['L_AVG_SELL_PRICE']
Covariates: ['CONST', 'ARTICLE_TIER_BETTER', 'ARTICLE_TIER_GOOD', 'ART_BRAND_TIER_PRICE_ENTRY', 'ART_BRAND_TIER_PRIVATE_LABEL', 'PACKAGING_BOX', 'PACKAGING_CAN', 'PACKAGING_PACK', 'ARTICLE_CAT_2_DRINKPAKJES', 'ARTICLE_CAT_2_FRUITDRANK', 'ARTICLE_CAT_2_IJSTHEE', 'ARTICLE_CAT_2_LIMONADE_SIROPEN', 'ARTICLE_CAT_2_SAPPEN_SMOOTHIES', 'ARTICLE_CAT_2_SINAS_LEMON_CASSIS', 'ARTICLE_CAT_2_SPECIAAL_FRIS', 'ARTICLE_CAT_2_SPORT_ENERGYDRINK', 'ARTICLE_CAT_2_WATER', 'PROMO_DUMMY', 'PPL2', 'PPL3', 'POSTPPL1', 'POSTPPL2', 'POSTPPL3', 'L_AVG_HIGH_TEMP', 'L_NR_ARTICLES_IN_CAT', 'L_NR_ARTICLES_IN_CAT_2', 'AVG_UNAVAILABILITY_PERC', 'ART_CONTENT_VOLUME', 'ART_IS_MULTIPACK', 'HOLIDAY', 'WEEK_NR']
Instrument variable(s): None
No. Observations: 12313

------------------ Score & algorithm ----

RFor

In [21]:
# Define what column to filter on
subset_col='ART_CAT_2'

# Get the unique values of the column that we want to use for dividing the subsets
unique_values = dmlset_lasso_train[subset_col].unique()
# Store the results of each subset in a dictionary
results5 = {}
for value in unique_values:
    print("Running for", value,"...")
    subset = dmlset_lasso_train[dmlset_lasso_train[subset_col] == value]
    # Perform the model operation on the subset
    obj_dml_data = dml.DoubleMLData(subset.drop(['ART_CAT_2','L_AVG_PURCHASE_PRICE'],axis=1), y_col='L_PRODUCT_ORDER_AMT', d_cols='L_AVG_SELL_PRICE', force_all_x_finite=True)
    dml_pliv_obj = dml.DoubleMLPLR(obj_dml_data, Learnerml_l,Learnerml_r)
    print(dml_pliv_obj.fit())
    result = dml_pliv_obj.coef
    # Store the result of the model operation in the dictionary
    results5[value] = result

# Print all the results
for key, value in results5.items():
    print(f"Result for value {key}: {value}")

Running for Fruitdrank ...
================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: L_PRODUCT_ORDER_AMT
Treatment variable(s): ['L_AVG_SELL_PRICE']
Covariates: ['CONST', 'ARTICLE_TIER_BETTER', 'ARTICLE_TIER_GOOD', 'ART_BRAND_TIER_PRICE_ENTRY', 'ART_BRAND_TIER_PRIVATE_LABEL', 'PACKAGING_BOX', 'PACKAGING_CAN', 'PACKAGING_PACK', 'ARTICLE_CAT_2_DRINKPAKJES', 'ARTICLE_CAT_2_FRUITDRANK', 'ARTICLE_CAT_2_IJSTHEE', 'ARTICLE_CAT_2_LIMONADE_SIROPEN', 'ARTICLE_CAT_2_SAPPEN_SMOOTHIES', 'ARTICLE_CAT_2_SINAS_LEMON_CASSIS', 'ARTICLE_CAT_2_SPECIAAL_FRIS', 'ARTICLE_CAT_2_SPORT_ENERGYDRINK', 'ARTICLE_CAT_2_WATER', 'PROMO_DUMMY', 'PPL2', 'PPL3', 'POSTPPL1', 'POSTPPL2', 'POSTPPL3', 'L_AVG_HIGH_TEMP', 'L_NR_ARTICLES_IN_CAT', 'L_NR_ARTICLES_IN_CAT_2', 'AVG_UNAVAILABILITY_PERC', 'ART_CONTENT_VOLUME', 'ART_IS_MULTIPACK', 'HOLIDAY', 'WEEK_NR']
Instrument variable(s): None
No. Observations: 12313

------------------ Score & algorithm ----

XGBoost

In [22]:
#Initialize XGBoost (Parameters Optimized Elsewhere)
xgbr_l=xgb.XGBRegressor(colsample_bytree= 1, gamma= 0, learning_rate= 0.2, max_depth= None, n_estimators= 500).fit(x_xgb_train,y_xgb_train)
xgbr_r=xgb.XGBRegressor(colsample_bytree= 1, gamma= 0, learning_rate= 0.2, max_depth= None, n_estimators= 500).fit(x_xgb_train,p_xgb_train)


In [23]:
# Define what column to filter on
subset_col='ART_CAT_2'

# Get the unique values of the column that we want to use for dividing the subsets
unique_values = dmlset_lasso_train[subset_col].unique()
# Store the results of each subset in a dictionary
results6 = {}
for value in unique_values:
    print("Running for", value,"...")
    subset = dmlset_lasso_train[dmlset_lasso_train[subset_col] == value]
    # Perform the model operation on the subset
    obj_dml_data = dml.DoubleMLData(subset.drop(['ART_CAT_2','L_AVG_PURCHASE_PRICE'],axis=1), y_col='L_PRODUCT_ORDER_AMT', d_cols='L_AVG_SELL_PRICE', force_all_x_finite=True)
    dml_pliv_obj = dml.DoubleMLPLR(obj_dml_data, xgbr_l,xgbr_r)
    print(dml_pliv_obj.fit())
    result = dml_pliv_obj.coef
    # Store the result of the model operation in the dictionary
    results6[value] = result

# Print all the results
for key, value in results6.items():
    print(f"Result for value {key}: {value}")

Running for Fruitdrank ...
================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: L_PRODUCT_ORDER_AMT
Treatment variable(s): ['L_AVG_SELL_PRICE']
Covariates: ['CONST', 'ARTICLE_TIER_BETTER', 'ARTICLE_TIER_GOOD', 'ART_BRAND_TIER_PRICE_ENTRY', 'ART_BRAND_TIER_PRIVATE_LABEL', 'PACKAGING_BOX', 'PACKAGING_CAN', 'PACKAGING_PACK', 'ARTICLE_CAT_2_DRINKPAKJES', 'ARTICLE_CAT_2_FRUITDRANK', 'ARTICLE_CAT_2_IJSTHEE', 'ARTICLE_CAT_2_LIMONADE_SIROPEN', 'ARTICLE_CAT_2_SAPPEN_SMOOTHIES', 'ARTICLE_CAT_2_SINAS_LEMON_CASSIS', 'ARTICLE_CAT_2_SPECIAAL_FRIS', 'ARTICLE_CAT_2_SPORT_ENERGYDRINK', 'ARTICLE_CAT_2_WATER', 'PROMO_DUMMY', 'PPL2', 'PPL3', 'POSTPPL1', 'POSTPPL2', 'POSTPPL3', 'L_AVG_HIGH_TEMP', 'L_NR_ARTICLES_IN_CAT', 'L_NR_ARTICLES_IN_CAT_2', 'AVG_UNAVAILABILITY_PERC', 'ART_CONTENT_VOLUME', 'ART_IS_MULTIPACK', 'HOLIDAY', 'WEEK_NR']
Instrument variable(s): None
No. Observations: 12313

------------------ Score & algorithm ----

Info obs categories

In [24]:
df['ART_CAT_2'].value_counts()

Fruitdrank               13694
Sappen & smoothies       11049
Speciaal fris             8867
Cola                      7938
Limonade & siropen        7700
Drinkpakjes               7352
Sinas, Lemon & Cassis     4906
Water                     4017
Sport- & energydrink      1206
Ijsthee                   1182
Name: ART_CAT_2, dtype: int64

# Residual analysis

Make predictions based on eqn (4.1)

In [26]:
#choose and fit model to use for predictions - this is PLR 1
lasso_l2 = LassoCV(fit_intercept=True, cv=10).fit(x_lasso_train,y_lasso_train)

In [29]:
# Define what column to filter on
subset_col='ART_CAT_2'

# Get the unique elasticity values for each subcategory
unique_values = {'Fruitdrank': , #input elasticities to use for prediction here
'Cola': ,
'Limonade & siropen': ,
'Water': ,
'Speciaal fris': ,
'Sappen & smoothies':,
'Drinkpakjes': ,
'Sinas, Lemon & Cassis': ,
'Ijsthee': ,
'Sport- & energydrink':}
# Store the results of each subset in a dictionary
results5 = {}
for key, value in unique_values.items():
    print("Running for", key,"...")
    subset = dmlset_lasso_train[dmlset_lasso_train[subset_col] == key]
    # Perform the  operation on the subset
    y_pred_lasso=lasso_l2.predict(subset.drop(['ART_CAT_2','L_AVG_PURCHASE_PRICE','L_AVG_SELL_PRICE','L_PRODUCT_ORDER_AMT'],axis=1)) 
    # Store the result of the model operation in the dictionary
    theta_lasso=value

    final_y_pred_lasso_train=(subset['L_AVG_SELL_PRICE'])*(theta_lasso)+y_pred_lasso
    lassormsetr=mean_squared_error(final_y_pred_lasso_train,subset['L_PRODUCT_ORDER_AMT'],squared=False)
    plt.figure(figsize=(10,10))
    sns.scatterplot(y=y_lasso_train,x=y_lasso_train-final_y_pred_lasso_train,label='Lasso')
    plt.xlabel('Prediction Residuals')
    plt.ylabel('Y')
    results5[key] = np.corrcoef(final_y_pred_lasso_train,subset['L_AVG_SELL_PRICE'])

# Print all the results
for key, value in results5.items():
    print(f"Result for value {key}: {value}")

SyntaxError: invalid syntax (2051730325.py, line 7)

In [ ]:
#make first stage estimated values trained on 2020-2022 set for both lasso and xgb
y_pred_lasso=lasso_l.predict(x_lasso_test) # models fitted on train set
p_pred_lasso=lasso_m.predict(x_lasso_test)

y_pred_lasso_train=lasso_l.predict(x_lasso_train) # models fitted on train set

#make final predictions based on estimated theta's and first stages
#use estimated theta's
theta_lasso=
theta_rfor=
theta_xgb=
#final estimated demand:
# old prediction formula (gave skewed realtion residuals/predictions)
# final_y_pred_lasso=(p_lasso_test-p_pred_lasso)*(theta_lasso)+y_pred_lasso
# final_y_pred_rfor=(p_lasso_test-p_pred_rfor)*(theta_rfor)+y_pred_rfor
# final_y_pred_xgb=(p_test-p_pred_xgb)*(theta_xgb)+y_pred_xgb 

final_y_pred_lasso_train=(p_lasso_train)*(theta_lasso)+y_pred_lasso_train


lassormsetr=mean_squared_error(final_y_pred_lasso_train,y_lasso_train,squared=False)

print("\nLasso RMSE on train dataset using estimated theta = ", lassormsetr, "\nLasso relative RMSE on train dataset using estimated theta = ", lassormsetr/y_lasso_train.mean())

PLR2?

In [ ]:
 {
'Fruitdrank': ,
'Cola': ,
'Limonade & siropen': ,
'Water': ,
'Speciaal fris': ,
'Sappen & smoothies': ,
'Drinkpakjes': ,
'Sinas, Lemon & Cassis': ,
'Ijsthee': ,
'Sport- & energydrink':
}